<a href="https://colab.research.google.com/github/hellwalker3/Vocal_separation_with_Demucs_and_Speaker_Conversion_with_so-vits-svc4.0/blob/main/Vocal_separation_with_Demucs_and_Speaker_Conversion_with_so_vits_svc4_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Mount google drive and select which directories to sync with google drive

from google.colab import drive
drive.mount("/content/drive")


In [ ]:
#@title install demucs code
!python3 -m pip install -U git+https://github.com/facebookresearch/demucs#egg=demucs

In [ ]:
#@title Settings of demucs
model = "htdemucs"
extensions = ["mp3", "wav", "ogg", "flac"]  # we will look for all those file types.
two_stems = None   # only separate one stems from the rest, for instance
# two_stems = "vocals"

# Options for the output audio.
mp3 = True
mp3_rate = 320
float32 = False  # output as float 32 wavs, unsused if 'mp3' is True.
int24 = False    # output as int24 wavs, unused if 'mp3' is True.
# You cannot set both `float32 = True` and `int24 = True` !!

In [ ]:
#@title Useful functions of demucs, don't forget to execute
import io
from pathlib import Path
import select
from shutil import rmtree
import subprocess as sp
import sys
from typing import Dict, Tuple, Optional, IO

def find_files(in_path):
    out = []
    for file in Path(in_path).iterdir():
        if file.suffix.lower().lstrip(".") in extensions:
            out.append(file)
    return out

def copy_process_streams(process: sp.Popen):
    def raw(stream: Optional[IO[bytes]]) -> IO[bytes]:
        assert stream is not None
        if isinstance(stream, io.BufferedIOBase):
            stream = stream.raw
        return stream

    p_stdout, p_stderr = raw(process.stdout), raw(process.stderr)
    stream_by_fd: Dict[int, Tuple[IO[bytes], io.StringIO, IO[str]]] = {
        p_stdout.fileno(): (p_stdout, sys.stdout),
        p_stderr.fileno(): (p_stderr, sys.stderr),
    }
    fds = list(stream_by_fd.keys())

    while fds:
        # `select` syscall will wait until one of the file descriptors has content.
        ready, _, _ = select.select(fds, [], [])
        for fd in ready:
            p_stream, std = stream_by_fd[fd]
            raw_buf = p_stream.read(2 ** 16)
            if not raw_buf:
                fds.remove(fd)
                continue
            buf = raw_buf.decode()
            std.write(buf)
            std.flush()

def separate(inp=None, outp=None):
    inp = inp or in_path
    outp = outp or out_path
    cmd = ["python3", "-m", "demucs.separate", "-o", str(outp), "-n", model]
    if mp3:
        cmd += ["--mp3", f"--mp3-bitrate={mp3_rate}"]
    if float32:
        cmd += ["--float32"]
    if int24:
        cmd += ["--int24"]
    if two_stems is not None:
        cmd += [f"--two-stems={two_stems}"]
    files = [str(f) for f in find_files(inp)]
    if not files:
        print(f"No valid audio files in {in_path}")
        return
    print("Going to separate the files:")
    print('\n'.join(files))
    print("With command: ", " ".join(cmd))
    p = sp.Popen(cmd + files, stdout=sp.PIPE, stderr=sp.PIPE)
    copy_process_streams(p)
    p.wait()
    if p.returncode != 0:
        print("Command failed, something went wrong.")



In [ ]:
#@title Install so-vits-svc-fork and its dependencies
#@markdown pip may fail to resolve dependencies and raise ERROR, but it can be ignored.
!python -m pip install -U pip wheel

%pip install -U ipython 

#@markdown Branch (for development)
BRANCH = "none" #@param {"type": "string"}
if BRANCH == "none":
    %pip install -U so-vits-svc-fork

!pip install pydub
!pip install yt-dlp

In [ ]:
#@title Utility function to download music
from yt_dlp import YoutubeDL

def download_audio(url, save_path, audio_format="mp3"):
    ydl_opts = {
        "format": "bestaudio/best",
        "outtmpl": save_path,
        "postprocessors": [{"key": "FFmpegExtractAudio", "preferredcodec": audio_format}],
    }

    with YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])

In [ ]:
#@title Required configuration Settings for Model, Song, and YouTube URL
# Set the model folder name and folder path
# The folder must include exclusively the G_*.pth file and the config.json file.
model_folder_name = ""
model_folder_path = f"/content/drive/MyDrive/{model_folder_name}"

# Set the song title and folder path
# The folder should contain the music content
# If this is the first time using this song, it will be downloaded from the provided youtube_url and separated
song_title = ""
song_folder_path = f"/content/drive/MyDrive/{song_title}"

# Set the YouTube URL for the song to be downloaded
youtube_url = ""



In [ ]:
#@title Demucs Vocal Separation, Conversion with SO-VITS-SVC 4.0, and Music Integration to Generate /content/output_vocals.mp3 (only vocals) and /content/output_music.mp3 (with bgm)
import os, glob, shutil
from pydub import AudioSegment
from IPython.display import Audio, display

def create_directories():
    os.makedirs(song_folder_path, exist_ok=True)
    os.makedirs(os.path.join(song_folder_path, "full"), exist_ok=True)

def download_audio_if_needed():
    save_path = os.path.join(song_folder_path, "full/music")
    if not os.path.exists(save_path + ".mp3"):
        download_audio(youtube_url, save_path)

def separate_audio():
    music_path = os.path.join(song_folder_path, "full")
    if not os.path.exists(music_path + "_sep"):
        separate(music_path, music_path + "_sep")

def get_source_path():
    music_path = os.path.join(song_folder_path, "full")
    for file in glob.glob(f"{os.path.join(music_path + '_sep', 'htdemucs/music')}/**"):
        if file[-10:] == "vocals.mp3":
            return file
    return None

def copy_config_and_model_files():
    config_suffix = ".json"
    config_file = None
    for file_name in os.listdir(model_folder_path):
        if file_name.endswith(config_suffix):
            config_file = file_name
            shutil.copy(os.path.join(model_folder_path, file_name), file_name)
            break
    if config_file is None:
        raise ValueError("Config file not found in the folder.")

    model_suffix = ".pth"
    target_model_file = None
    for file_name in os.listdir(model_folder_path):
        if file_name.endswith(model_suffix):
            target_model_file = file_name
            shutil.copy(os.path.join(model_folder_path, file_name), file_name)
            break
    if target_model_file is None:
        raise ValueError("Model file not found in the folder.")
    return config_file, target_model_file

def infer_svc(source_path, target_model_file, config_file):
    # need to improve
    !svc infer {source_path} -m  /content/{target_model_file} -c /content/{config_file} -fm crepe -na 
    !mv {source_path[:-4]+".out.mp3"} /content/output_vocals.mp3

def get_full_music_list():
    music_path = os.path.join(song_folder_path, "full")
    return [file_path for file_path in glob.glob(os.path.join(music_path+'_sep', 'htdemucs/music', '**')) if file_path[-10:] != 'vocals.mp3'] + ["/content/output_vocals.mp3"]

def mix_audio_segments(full_music_list):
    audio_segments = [AudioSegment.from_file(audio_file, format='mp3').set_frame_rate(44100) for audio_file in full_music_list]
    mixed = audio_segments[0]
    for audio_segment in audio_segments[1:]:
        mixed = mixed.overlay(audio_segment)
    return mixed

def export_and_display_audio(mixed):
    integrated_audio_path = "/content/output_music.mp3"
    mixed.export(integrated_audio_path, format='mp3')
    display(Audio(integrated_audio_path, autoplay=True))

create_directories()
download_audio_if_needed()
separate_audio()

source_path = get_source_path()
config_file, target_model_file = copy_config_and_model_files()
infer_svc(source_path, target_model_file, config_file)

full_music_list = get_full_music_list()
mixed_audio = mix_audio_segments(full_music_list)
export_and_display_audio(mixed_audio)

In [ ]:
#@title If the audio quality is poor, you may want to consider adjusting the arguments for the 'svc infer' command based on the suggestions below.
!svc infer -h

In [ ]:
#@title Download generated file
from google.colab import files
files.download("/content/output_music.mp3")